# CODEBERT 모델로 교체 사용

In [ ]:
!pip install transformers seqeval[gpu]

In [ ]:
!pip install datasets seqeval torch scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
from seqeval.metrics import classification_report

In [ ]:
# 작업 환경 확인하기 - 가급적이면 GPU
from torch import cuda

# 모델이 GPU에서 실행 중이면, GPU로 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
1. codebert 기반 학습 데이터 준비

In [ ]:
import pandas as pd
from transformers import RobertaTokenizer

# 📌 데이터 경로 설정
TRAIN_DATA_PATH = "/content/balanced_ner_dataset.csv"
OUTPUT_TRAIN_PATH = "/content/ner_codebert_training_data.csv"

# 🔹 CodeBERT 토크나이저 로드
model_name = "microsoft/codebert-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)

# 🔹 데이터 로드
df = pd.read_csv(TRAIN_DATA_PATH, encoding="utf-8-sig")

# 🔹 WordPiece 토큰화 및 NER 라벨 적용
wordpiece_tokens_list = []
ner_labels_list = []

for _, row in df.iterrows():
    words = [row["name"].lower()]  # name 컬럼 (기본 단어)

    for word in words:
        tokens = tokenizer.tokenize(word)  # CodeBERT 토큰화
        ner_label = row["ner_label"]  # NER 라벨 적용

        wordpiece_tokens_list.append(" ".join(tokens))
        ner_labels_list.append(" ".join([ner_label] * len(tokens)))  # 모든 워드피스에 동일한 라벨 적용

# 🔹 새로운 데이터프레임 생성
train_df = pd.DataFrame({
    "wordpiece_tokens": wordpiece_tokens_list,
    "ner_labels": ner_labels_list
})

# 🔹 저장
train_df.to_csv(OUTPUT_TRAIN_PATH, index=False, encoding="utf-8-sig")
print(f"✅ CodeBERT 기반 NER 훈련 데이터 생성 완료! 파일 저장: {OUTPUT_TRAIN_PATH}")


✅ CodeBERT 기반 NER 훈련 데이터 생성 완료! 파일 저장: /content/ner_codebert_training_data.csv


In [ ]:
# RoBERTa는 tokenizer(is_split_into_words=True) 옵션이 없음, 따라서 WordPiece 토큰화 방식과 다름.
데이터 전처리 시, WordPiece 기반이 아니라 띄어쓰기 기준 토큰화를 잘 처리해야 함.


In [ ]:
2. 데이터 전처리 및 학습 데이터셋 변환

In [ ]:
import torch
from datasets import Dataset

# 🔹 NER 태그 정의
ner_labels = ["O", "B-LANG", "B-FRAME", "B-LIB", "B-TOOL"]
label_map = {label: idx for idx, label in enumerate(ner_labels)}

# 🔹 데이터 전처리 함수
def preprocess_data(examples):
    tokens = examples["wordpiece_tokens"].split()
    labels = examples["ner_labels"].split()

    # ✅ 토큰화 실행
    inputs = tokenizer(
        tokens,
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=512
    )

    # ✅ labels의 길이를 input_ids와 동일하도록 설정
    input_length = len(inputs["input_ids"])
    label_ids = [label_map.get(label, label_map["O"]) for label in labels]
    label_ids += [label_map["O"]] * (input_length - len(label_ids))
    label_ids = label_ids[:input_length]

    inputs["labels"] = label_ids
    return inputs

# 🔹 Dataset 변환 및 전처리
dataset = Dataset.from_pandas(train_df)
dataset = dataset.map(preprocess_data)

# 🔹 훈련 & 평가 데이터 분할
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset, eval_dataset = train_test_split["train"], train_test_split["test"]


Map:   0%|          | 0/1141 [00:00<?, ? examples/s]

In [ ]:
3. codebert 기반 ner 학습 진행

In [ ]:
from transformers import RobertaForTokenClassification, Trainer, TrainingArguments

# 🔹 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir="./codebert_ner_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10
)

# 🔹 CodeBERT 모델 초기화 (NER 태그 개수 지정)
model = RobertaForTokenClassification.from_pretrained(model_name, num_labels=len(ner_labels))

# 🔹 Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# 🔹 모델 학습 시작
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-8-43f4169f14b6>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cgygy8989 (cgygy8989-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.018500,0.015097
2,0.014400,0.013791
3,0.013500,0.013214
4,0.013200,0.013132
5,0.013200,0.013031


TrainOutput(global_step=570, training_loss=0.02816638660274054, metrics={'train_runtime': 703.5776, 'train_samples_per_second': 6.481, 'train_steps_per_second': 0.81, 'total_flos': 1191545527910400.0, 'train_loss': 0.02816638660274054, 'epoch': 5.0})

In [ ]:
import os

model_path = "./codebert_ner_model"
print("모델 저장 경로 확인:", os.path.exists(model_path))
print("모델 폴더 내용물 확인:", os.listdir(model_path) if os.path.exists(model_path) else "폴더 없음")


모델 저장 경로 확인: True
모델 폴더 내용물 확인: ['checkpoint-456', 'checkpoint-228', 'checkpoint-114', 'checkpoint-570', 'checkpoint-342']


In [ ]:
from transformers import RobertaForTokenClassification, Trainer, TrainingArguments


trainer.save_model("./codebert_ner_model")  # 명시적으로 모델 저장
tokenizer.save_pretrained("./codebert_ner_model")
print("✅ CodeBERT 모델 저장 완료!")


✅ CodeBERT 모델 저장 완료!


In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForTokenClassification

# ✅ 모델 저장 경로 확인
MODEL_PATH = "./codebert_ner_model"
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"❌ 모델 경로가 존재하지 않습니다: {MODEL_PATH}")

# ✅ 저장된 모델 불러오기
tokenizer = RobertaTokenizer.from_pretrained(MODEL_PATH)
model = RobertaForTokenClassification.from_pretrained(MODEL_PATH)

# ✅ GPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("✅ CodeBERT 모델 로드 완료!")


✅ CodeBERT 모델 로드 완료!


In [ ]:
4. codebert 기반 예측 수행

In [ ]:
import pandas as pd
import os
import torch
from transformers import RobertaTokenizer, RobertaForTokenClassification
from seqeval.metrics import classification_report

# 📌 예측할 데이터 경로
TEST_DATA_PATH = "/content/combined_morpheme.csv"
MODEL_PATH = "./codebert_ner_model"

# 🔹 학습된 모델 로드
tokenizer = RobertaTokenizer.from_pretrained(MODEL_PATH)
model = RobertaForTokenClassification.from_pretrained(MODEL_PATH)

# 🔹 GPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 🔹 NER 태그 정의
ner_labels = ["O", "B-LANG", "B-FRAME", "B-LIB", "B-TOOL"]

# ✅ 모델 예측 수행 함수
def predict_ner(text):
    tokens = tokenizer.tokenize(text)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=2)
    predicted_labels = [ner_labels[idx.item()] for idx in predictions[0]]

    return tokens, predicted_labels

# ✅ 테스트 데이터 로드
df_test = pd.read_csv(TEST_DATA_PATH, encoding="utf-8-sig")

# ✅ 모든 데이터에 대해 예측 수행
df_test["tokens"], df_test["predicted_labels"] = zip(*df_test["combined_morpheme"].apply(predict_ner))

# 🔹 평가 수행 (정답과 비교)
if "ner_label" in df_test.columns:  # 정답이 포함된 경우
    true_labels = df_test["ner_label"].apply(lambda x: x.split())  # 실제 정답 라벨
    pred_labels = df_test["predicted_labels"].apply(lambda x: x)

    # 🔥 평가 지표 출력
    print("\n🔹 NER 평가 결과:\n", classification_report(true_labels.tolist(), pred_labels.tolist(), digits=4))

# ✅ 예측 결과 저장
OUTPUT_FILE = "/content/codebert_ner_predicted_results.csv"
df_test.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
print(f"✅ CodeBERT 예측 완료 및 저장: {OUTPUT_FILE}")


✅ CodeBERT 예측 완료 및 저장: /content/codebert_ner_predicted_results.csv


# 데이터 자체에 문제, NER 균등 작업 다시 하기

In [ ]:
import pandas as pd

# 균형 잡힌 NER 데이터 로드
df = pd.read_csv("/content/balanced_ner_dataset.csv", encoding="utf-8-sig")

# NER 태그 개수 확인
ner_counts = df["ner_label"].value_counts()
print("✅ 현재 NER 태그별 개수:\n", ner_counts)


✅ 현재 NER 태그별 개수:
 ner_label
B-TOOL     463
B-FRAME    291
B-LIB      198
B-LANG     189
Name: count, dtype: int64


In [ ]:
# 원본 technical_element_labeled2에서 b-tool만 다시 가져와서 복붙
import pandas as pd

# 균형 잡힌 NER 데이터 로드
df = pd.read_csv("/content/balanced_ner_dataset(Original tool).csv")

# NER 태그 개수 확인
ner_counts = df["ner_label"].value_counts()
print("✅ 현재 NER 태그별 개수:\n", ner_counts)


In [ ]:
import pandas as pd
import chardet

# 파일 경로
file_path = "/content/balanced_ner_dataset(Original tool).csv"

# 🔹 파일의 인코딩 확인
with open(file_path, "rb") as f:
    result = chardet.detect(f.read(100000))  # 100000 바이트 샘플링
    detected_encoding = result["encoding"]

print(f"✅ 감지된 인코딩: {detected_encoding}")

# 🔹 감지된 인코딩으로 CSV 파일 로드
df = pd.read_csv(file_path, encoding=detected_encoding)

# 🔹 NER 태그 개수 확인
ner_counts = df["ner_label"].value_counts()
print("✅ 현재 NER 태그별 개수:\n", ner_counts)


In [ ]:
# 수작업으로 복붙하면 인코딩 오류가 나므로 코딩으로 해결한다.

In [ ]:
/content/technical_element_labeled2.csv

In [ ]:
import pandas as pd

# 📌 원본 파일 경로
input_file = "/content/technical_element_labeled2.csv"
output_file = "/content/b_tool_only.csv"  # B-TOOL 항목만 저장할 파일

# 🔹 CSV 파일 로드
df = pd.read_csv(input_file, encoding="utf-8-sig")

# 🔹 'B-TOOL' 태그만 필터링
df_b_tool = df[df["ner_label"] == "B-TOOL"]

# 🔹 결과 저장
df_b_tool.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✅ B-TOOL 태그만 포함된 데이터 저장 완료! 저장 경로: {output_file}")


✅ B-TOOL 태그만 포함된 데이터 저장 완료! 저장 경로: /content/b_tool_only.csv


In [ ]:
print(f"🔹 추출된 B-TOOL 개수: {df_b_tool.shape[0]}개")
print(df_b_tool.head())  # 상위 5개 데이터 출력


🔹 추출된 B-TOOL 개수: 141개
     seq category      name ner_label
137  137     tool      agit    B-TOOL
138  138     tool   airflow    B-TOOL
139  139     tool   ansible    B-TOOL
140  140     tool     arcus    B-TOOL
141  141     tool  arangodb    B-TOOL


In [ ]:
위에서 추출한 141개 툴은 반드시 들어가야하는 본래 기술 사전이다. 증강한 balance_ner.csv 파일에 해당 기술 사전은 필수로 들어가도록 하고 나머지 3개의 ner 개수와 균등 작업을 다시 해야한다. 200개 내외


In [ ]:
import pandas as pd

# 📌 파일 경로 설정
original_file = "/content/balanced_ner_dataset.csv"  # 기존 균형 잡힌 데이터셋
b_tool_file = "/content/b_tool_only.csv"  # 필수 유지해야 하는 B-TOOL 데이터
output_file = "/content/final_balanced_ner_dataset.csv"  # 최종 균형 잡힌 데이터셋

# 🔹 CSV 파일 로드
df_original = pd.read_csv(original_file, encoding="utf-8-sig")
df_b_tool = pd.read_csv(b_tool_file, encoding="utf-8-sig")

# 🔹 태그별 개수 확인
tag_counts = df_original["ner_label"].value_counts()
print("🔹 기존 데이터 NER 태그별 개수:\n", tag_counts)

# 🔹 목표 개수 설정 (B-TOOL 필수 데이터에 맞춰 균등하게)
target_count = tag_counts["B-FRAME"]  # 291개

# 🔹 B-TOOL 처리: 필수 데이터 유지 + 부족한 개수 샘플링
b_tool_fixed = df_b_tool.copy()  # 필수 포함 B-TOOL 데이터 (141개)
needed_b_tool_samples = target_count - len(b_tool_fixed)  # 부족한 개수 계산 (291 - 141 = 150개)
sampled_b_tool = df_original[df_original["ner_label"] == "B-TOOL"].sample(n=needed_b_tool_samples, replace=True, random_state=42)
balanced_b_tool = pd.concat([b_tool_fixed, sampled_b_tool], ignore_index=True)

# 🔹 다른 태그 균등 샘플링
balanced_df = balanced_b_tool.copy()  # 초기값으로 B-TOOL 추가
for tag in ["B-FRAME", "B-LIB", "B-LANG"]:
    sampled_data = df_original[df_original["ner_label"] == tag].sample(n=target_count, replace=True, random_state=42)
    balanced_df = pd.concat([balanced_df, sampled_data], ignore_index=True)

# ✅ 최종 태그 개수 확인
final_tag_counts = balanced_df["ner_label"].value_counts()
print("✅ 균형 조정 후 NER 태그별 개수:\n", final_tag_counts)

# 📌 균형 잡힌 데이터 저장
balanced_df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"✅ 균형 잡힌 데이터 저장 완료! 저장 경로: {output_file}")


🔹 기존 데이터 NER 태그별 개수:
 ner_label
B-TOOL     463
B-FRAME    291
B-LIB      198
B-LANG     189
Name: count, dtype: int64
✅ 균형 조정 후 NER 태그별 개수:
 ner_label
B-TOOL     291
B-FRAME    291
B-LIB      291
B-LANG     291
Name: count, dtype: int64
✅ 균형 잡힌 데이터 저장 완료! 저장 경로: /content/final_balanced_ner_dataset.csv
